
# SimSwap for videos

Reference: [my changes to the official notebook](https://gist.github.com/woctezuma/78a98b73cbba8cba478d99c8c50bc359)

## Prepare code

In [ ]:
%cd /content
!git clone https://github.com/woctezuma/SimSwap.git
%cd /content/SimSwap/
!git checkout upgrade-insightface

In [ ]:
!pip install insightface==0.7.3 onnxruntime moviepy > /dev/null
!pip install imageio==2.34.0 > /dev/null

## Prepare models

In [ ]:
%cd /content/SimSwap

In [ ]:
!wget https://github.com/woctezuma/SimSwap-colab/releases/download/antelope/antelope.zip
!wget -P ./arcface_model https://github.com/woctezuma/SimSwap-colab/releases/download/1.0/arcface_checkpoint.tar
!wget https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip
!wget -P ./parsing_model/checkpoint https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth
!wget https://github.com/neuralchen/SimSwap/releases/download/512_beta/512.zip

In [ ]:
y!unzip ./checkpoints.zip  -d ./checkpoints

!unzip 512.zip -d ./checkpoints

!unzip antelope.zip -d ./insightface_func/models/

## Prepare data

### Download

In [ ]:
%cd /content

!curl https://i.imgur.com/iQtmj1N.png -o photo.png
!curl http://i.imgur.com/yCD2Uxm.gif -o video.gif

In [ ]:
input_image_fname = '/content/photo.png'
input_video_fname = '/content/video.gif'

## Run

### Official code

In [ ]:
%cd /content/SimSwap/

In [ ]:
import cv2
import torch
import fractions
import numpy as np
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
from models.models import create_model
from options.test_options import TestOptions
from insightface_func.face_detect_crop_multi import Face_detect_crop
from util.videoswap import video_swap
from util.add_watermark import watermark_image

In [ ]:
transformer = transforms.Compose([
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

transformer_Arcface = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

detransformer = transforms.Compose([
        transforms.Normalize([0, 0, 0], [1/0.229, 1/0.224, 1/0.225]),
        transforms.Normalize([-0.485, -0.456, -0.406], [1, 1, 1])
    ])

In [ ]:
# If the algorithm misses some faces, you could lower the detection threshold.
# Reference: https://github.com/neuralchen/SimSwap/issues/39#issuecomment-873758730

det_thresh = 0.6

# You could also decrease the image size used for face detection:

det_size = (640,640)

In [ ]:
# Either 224 (okay) or 512 (experimental)
crop_size = 224

In [ ]:
from torch.serialization import add_safe_globals
from torch.nn import Conv2d
from models.arcface_models import ResNet  # arcface 모델이면 ResNet도 필요할 수 있음

add_safe_globals([Conv2d, ResNet])


In [ ]:
# ArcFace 가중치 다운로드 (SimSwap 공식 제공)
!mkdir -p arcface_model
!wget -O ./arcface_model/arcface_checkpoint.tar https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar


In [ ]:
from torch.serialization import add_safe_globals
from torch.nn import (
    Conv2d, BatchNorm2d, BatchNorm1d, PReLU, MaxPool2d, Sequential,
    ReLU, Linear, AdaptiveAvgPool2d, Dropout, Module, Sigmoid
)
from models.arcface_models import IRBlock, SEBlock

add_safe_globals([
    Conv2d, BatchNorm2d, BatchNorm1d, PReLU, MaxPool2d, Sequential,
    ReLU, Linear, AdaptiveAvgPool2d, Dropout, Module,
    IRBlock, SEBlock, Sigmoid
])

from models.arcface_models import IRBlock  # arcface에서 사용하는 커스텀 블록
opt = TestOptions()
opt.initialize()
opt.parser.add_argument('-f') ## dummy arg to avoid bug
opt = opt.parse()
opt.pic_a_path = '/content/0544_00.png' ## or replace it with image from your own google drive
opt.video_path = '/content/target.mp4' ## or replace it with video from your own google drive
opt.output_path = '/content/output.mp4'
opt.temp_path = './tmp'
opt.Arc_path = './arcface_model/arcface_checkpoint.tar'
opt.isTrain = False
opt.no_simswaplogo = True
opt.use_mask = True
opt.crop_size = crop_size

if crop_size == 512:
    opt.which_epoch = 550000
    opt.name = '512'
    mode = 'ffhq'
else:
    mode = 'None'

torch.nn.Module.dump_patches = True
model = create_model(opt)
model.eval()


app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=det_thresh, det_size=det_size, mode=mode)

pic_a = opt.pic_a_path
# img_a = Image.open(pic_a).convert('RGB')
img_a_whole = cv2.imread(pic_a)
img_a_align_crop, _ = app.get(img_a_whole,crop_size)
img_a_align_crop_pil = Image.fromarray(cv2.cvtColor(img_a_align_crop[0],cv2.COLOR_BGR2RGB))
img_a = transformer_Arcface(img_a_align_crop_pil)
img_id = img_a.view(-1, img_a.shape[0], img_a.shape[1], img_a.shape[2])

# convert numpy to tensor
img_id = img_id.cuda()

#create latent id
img_id_downsample = F.interpolate(img_id, size=(112,112))
latend_id = model.netArc(img_id_downsample)
latend_id = F.normalize(latend_id, p=2, dim=1)

try:
  video_swap(opt.video_path, latend_id, model, app, opt.output_path,temp_results_dir=opt.temp_path,\
             no_simswaplogo=opt.no_simswaplogo,use_mask=opt.use_mask,crop_size=crop_size)
except IndexError:
  print('[error] This is most likely due to the absence of audio from a GIF input.')

In [ ]:
# ▶️ 필수: FFmpeg 설치
!apt-get update -y && apt-get install -y ffmpeg

# 📁 입력 파일 경로 (SimSwap 결과)
input_path = "/content/output.mp4"  # SimSwap 결과 영상

# 1️⃣ 노이즈 제거
!ffmpeg -y -i "{input_path}" -vf "hqdn3d=4.0:3.0:6.0:4.5" -c:a copy /content/output_denoised.mp4
# 3️⃣ 프레임 고정 & 고화질 인코딩
!ffmpeg -y -i /content/output_sharp.mp4 -filter:v "fps=30" -c:v libx264 -preset slow -crf 18 -pix_fmt yuv420p /content/final_output.mp4

# ✅ 최종 출력
from IPython.display import HTML
from base64 import b64encode

mp4 = open("/content/final_output.mp4", 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=640 controls><source src="{data_url}" type="video/mp4"></video>')
